### Kaden Garletts
### Date start: August 5th 2025, Date due: August 19th 2025 

  In core.ipynb, the conduction of webscraping of www.hackettstownbid.com, is used to perform Data Analysis used to answer several questions related to the planning of hackettstown busnisses, and to identify potential factors in business decisions. 

### The questions:
1. What type of businesses are the most popular in Hackettstown? "(ie. count per category compared to other categories)

2. Which types are the least popular?

3. Which businesses are closest to eachother

4. Which businesses are closest to the main roads

5. Which busineses are closest to businesses of related types? (using google maps API for matrix distance).

# Part 0.2: Exploratory Analysis of the Dataset Milestone #2

1. Becuase I chose to webscrape, its far harder to perform exploratory data analysis. 

a.describe(): There will not be a mean,std,min, or IQR --> but there will be the distance matrix

b.columns(): The columns are Restaurants, Community, Services, Shopping, and Entertainment/Gaming which have, names, and locations for rows

c.shape(): The shape of the data should be in 3 dimensions, as each row will be a list with name, and location

d.dtypes(): The datatypes should be float and string 

e.head(),tail(),sample() an example would be 'Mama's Cafe Baci', 40.8380252,-74.8263654 

f.info() 2 columns, x rows (probably ~50 per table) string, and float variable types.

# Part 0.3: Data Cleaning Report MileStone #3

For the first part of cleaning the data, I attempted to retrieve all of the names from each of the websites using beautiful soup. The first error I encountered when analysing my data was that there were missing names when I expected otherwise. To solve this I went into the HTML, and discovered that there were discrepancies on wether strong> tags were used or b> was used although they appeared the same. Solving this allowed for the full collection of names. 

Secondly, the adresses of each location are stored in the HTML aswell. I had issues reaching this one becuase they were stored in basic hyperlink tags without a coordinated identifier. I could have selected all of the hyper-links from an upper class, but It was diffiucult to access as all of the sub-classes had varying names. I discovered that becuase all of the sub-classes were similar in that they ended with thisClassx where x was a different number I could just iterate through every single class in order to store the links in a list. 

Further, the links are not what I needed, but rather the location, the initial plan was to just retrieve the coordiinates using the google map HTML, but I discovered that the extended link contains the coordinates. The issue is that the HackettstownBID website only provides the shortened link. To solve this, I am using requests to retrive the full URL from Google, but I am currently attempting to collect the data in a way that Google considers to not be bot activity. 

Afterwords, I will compare the length of the retirved data to the names to see if everything was collected appropriatly.

### Part 1: Webscraping https://hackettstownbid.com, and performing identifying measures to determine the categorization of businesses.

In [16]:
# Project imports

from bs4 import BeautifulSoup #| beautifulsoup4 used for parsing HTML
import requests as req #| requests used for making HTTP requests
import re #| re used for regular expressions
import seaborn as sns #| seaborn used for data visualization
import matplotlib.pyplot as plt #| matplotlib used for plotting graphs
import pandas as pd #| pandas used for data manipulation and analysis
import numpy as np #| numpy used for numerical operations
from geopy.distance import geodesic, great_circle #used to calculate geo-distance
import time #| used to pause the system breifly
from dotenv import load_dotenv # used to store sensitive info
import os #| Used to interact with the operating system
import pickle #| used to store data between runs
import random

load_dotenv() #loading sensitive info

True

In [17]:
# Project URL's

urlHackettstownResaurants = "https://hackettstownbid.com/explore/restaurants/"
urlHackettstownServices = "https://hackettstownbid.com/explore/services/"
urlHackettstownShopping = "https://hackettstownbid.com/explore/shopping/"
urlHackettstownEntertainmentGaming = "https://hackettstownbid.com/explore/entertainment-gaming/"
urlHackettstownCommunity = "https://hackettstownbid.com/explore/community/"

# Any businesses that are not included on the above pages are not registred with the Hackettstown Business Improvement Discrict

In [ ]:
# Using requests to fetch the HTML content of the pages

responseHackettstownResaurants = req.get(urlHackettstownResaurants)
responseHackettstownServices = req.get(urlHackettstownServices)
responseHackettstownShopping = req.get(urlHackettstownShopping)
responseHackettstownEntertainmentGaming = req.get(urlHackettstownEntertainmentGaming)
responseHackettstownCommunity = req.get(urlHackettstownCommunity)

In [ ]:
#parsing the html content into text through requests .text object

htmlContentHackettstownResaurants = responseHackettstownResaurants.text
htmlContentHackettstownServices = responseHackettstownServices.text
htmlContentHackettstownShopping = responseHackettstownShopping.text
htmlContentHackettstownEntertainmentGaming = responseHackettstownEntertainmentGaming.text
htmlContentHackettstownCommunity = responseHackettstownCommunity.text

In [ ]:
#creating BeautifulSoup objects for each html
soupHackettstownRestaurants = BeautifulSoup(htmlContentHackettstownResaurants, 'html.parser')
soupHackettstownServices = BeautifulSoup(htmlContentHackettstownServices, 'html.parser')
soupHackettstownShopping = BeautifulSoup(htmlContentHackettstownShopping, 'html.parser')
soupHackettstownEntertainmentGaming = BeautifulSoup(htmlContentHackettstownEntertainmentGaming, 'html.parser')
soupHackettstownCommunity = BeautifulSoup(htmlContentHackettstownCommunity, 'html.parser')

# Part 2: Extracting the content and storing the data into dataframe/s

In [ ]:
# Extracting the names of the businesses from the HTML content


def getBusinessNames(soupHackettstown):
    # https://stackoverflow.com/questions/68144219/how-to-scrape-text-from-strong-tag used to learn how to scrape from a strong tag
    strongHackettstown = [[i.get_text(strip=True) for i in div.find_all("strong")]
        for div in soupHackettstown.find_all(
            "div",
            class_="fusion-column-wrapper fusion-column-has-shadow fusion-flex-justify-content-flex-start fusion-content-layout-column"
        )]
    bHackettstown = [[i.get_text(strip=True) for i in div.find_all("b")]
        for div in soupHackettstown.find_all(
            "div",
            class_="fusion-column-wrapper fusion-column-has-shadow fusion-flex-justify-content-flex-start fusion-content-layout-column"
        )]
    strongHackettstown = [hList for hList in strongHackettstown if hList] #Assigns the var to only items that are lists
    strongHackettstown.pop(0) # Removes the first item which says "call us"
    bHackettstown = [hList for hList in bHackettstown if hList]
    bHackettstown.pop(0)
    namesHackettstown = [item for slist in strongHackettstown for item in slist] + [item for slist in bHackettstown for item in slist]
    return namesHackettstown

namesHackettstownRestaurants = getBusinessNames(soupHackettstownRestaurants)
namesHackettstownCommunity = getBusinessNames(soupHackettstownCommunity)
namesHackettstownEntertainmentGaming = getBusinessNames(soupHackettstownEntertainmentGaming)    
namesHackettstownServices = getBusinessNames(soupHackettstownServices)
namesHackettstownShopping = getBusinessNames(soupHackettstownShopping)



In [ ]:
#Extracts the links of the businesses locations from the HTML

def getBusinessLocations(soupHackettstown):
    fusionTextNumber = 1  # increments for the naming convention of the divs
    classHackettstown = []  # List to store the divs with the class fusion-text-fusionTextNumber
    linksHackettstown = []  # List to store the links
    soupIteration = True

    while soupIteration:
        classHackettstown.append(
            soupHackettstown.find_all(
                "div", {"class": "fusion-text fusion-text-" + str(fusionTextNumber)}
            )
        )  # Finds all divs with the class fusion-text-fusionTextNumber
        fusionTextNumber += 1  # Increments the fusionTextNumber to find the next div
        if fusionTextNumber >= 20:
            if (
                classHackettstown[-1] == []
                and classHackettstown[-2] == []
                and classHackettstown[-3] == []
            ):  # If the last three items in classHackettstown are empty the loop ends
                soupIteration = False

    # Flatten the list of lists into a single list
    classHackettstown = [item for sublist in classHackettstown for item in sublist]

    for i in classHackettstown:
        linkTags = i.find_all("a")  # Finds all <a> tags in the divs
        for j in linkTags:  # Iterates through each <a> tag
            href = j.get("href")  # Gets the href attribute of the <a> tag
            if href and (
                href.startswith("https://goo.gl/maps/")
                or href.startswith("https://maps.app.goo.gl/")
            ):  # Checks if the link starts with the Google Maps URL
                linksHackettstown.append(href)  # Adds the link to the list if it does

    # Used to extend the URLs that are shortened by the hyperlinks
    # I was able to do this thanks to Stack Overflow

    # splitting the links to make storage easier...
    if len(linksHackettstown) > 100:
        linksTo100 = linksHackettstown[:100]
        linksPast100 = linksHackettstown[100:]
        # We'll create pickles for these after expansion if missing (see below)

    countTo49 = 0
    countTo100 = 0
    countLocationsCaptured = 0
    randTime = (random.random() + 3)

    for i in range(len(linksHackettstown)):
        print("Attempting session")
        session = req.Session()  # so connections are recycled
        print("Attempting Head")
        try:
            resp = session.head(linksHackettstown[i], allow_redirects=True, timeout=30)
            print("Attempt resp.url")
            linksHackettstown[i] = resp.url
        except Exception as e:
            # If head fails, keep the original short link so you can inspect later
            print(f"Head failed for index {i} ({linksHackettstown[i]}): {e}")
            # don't overwrite the entry; leave it as-is (short link)
        randTime = (random.random() + 3)
        time.sleep(randTime)  # Sleep to avoid Google getting mad...
        countTo49 += 1
        countTo100 += 1
        countLocationsCaptured += 1
        print(countLocationsCaptured)
        if countTo49 == 20:
            print("sleeping for batch pause")
            time.sleep(125 + randTime)
            countTo49 = 0
        if countTo100 == 100:
            print("sleeping for 100 pause")
            time.sleep(300 + randTime)
            countTo100 = 0

    # After expanding all links, if >100 split into two service pickles (create only if missing)
    if len(linksHackettstown) > 100:
        svc1_file = "hackettstown_locations_services1.pkl"
        svc2_file = "hackettstown_locations_services2.pkl"
        linksTo100 = linksHackettstown[:100]
        linksPast100 = linksHackettstown[100:]

        # create services1 if it does not exist
        try:
            with open(svc1_file, "rb") as f:
                # exists already — leave it
                pass
        except FileNotFoundError:
            try:
                with open(svc1_file, "wb") as f:
                    pickle.dump(linksTo100, f)
                print(f"Saved first 100 links to {svc1_file}")
            except Exception as e:
                print("Failed to save services1 pickle:", e)

        # create services2 if it does not exist
        try:
            with open(svc2_file, "rb") as f:
                # exists already — leave it
                pass
        except FileNotFoundError:
            try:
                with open(svc2_file, "wb") as f:
                    pickle.dump(linksPast100, f)
                print(f"Saved remaining links to {svc2_file}")
            except Exception as e:
                print("Failed to save services2 pickle:", e)

    time.sleep(20 + randTime)
    print("Sleeping")
    return linksHackettstown


# Attempting to load the pickle, and if it does not --> function call
try:
    with open("hackettstown_locations_restaurants.pkl", "rb") as f:
        print("Loading pickle...")
        locationHackettstownRestaurants = pickle.load(f)
except FileNotFoundError:
    print("Could not find file... attempting to retrieve data")
    locationHackettstownRestaurants = getBusinessLocations(soupHackettstownRestaurants)
    print("Data retrieved! now adding to collection")
    with open("hackettstown_locations_restaurants.pkl", "wb") as f:
        pickle.dump(locationHackettstownRestaurants, f)

try:
    with open("hackettstown_locations_community.pkl", "rb") as f:
        print("Loading pickle...")
        locationHackettstownCommunity = pickle.load(f)
except FileNotFoundError:
    print("Could not find file... attempting to retrieve data")
    locationHackettstownCommunity = getBusinessLocations(soupHackettstownCommunity)
    print("Data retrieved! now adding to collection")
    with open("hackettstown_locations_community.pkl", "wb") as f:
        pickle.dump(locationHackettstownCommunity, f)

try:
    with open("hackettstown_locations_entertainmentgaming.pkl", "rb") as f:
        print("Loading pickle...")
        locationHackettstownEntertainmentGaming = pickle.load(f)
except FileNotFoundError:
    print("Could not find file... attempting to retrieve data")
    locationHackettstownEntertainmentGaming = getBusinessLocations(soupHackettstownEntertainmentGaming)
    print("Data retrieved! now adding to collection")
    with open("hackettstown_locations_entertainmentgaming.pkl", "wb") as f:
        pickle.dump(locationHackettstownEntertainmentGaming, f)

# SERVICES: attempt to load services1, if missing call getBusinessLocations once and create the missing pickles
all_services = None

try:
    with open("hackettstown_locations_services1.pkl", "rb") as f:
        print("Loading services1 pickle...")
        locationHackettstownServices1 = pickle.load(f)
except FileNotFoundError:
    print("services1 pickle missing; retrieving services links now...")
    all_services = getBusinessLocations(soupHackettstownServices)
    # try to load services1 again (function may have created it), otherwise create from all_services
    try:
        with open("hackettstown_locations_services1.pkl", "rb") as f:
            locationHackettstownServices1 = pickle.load(f)
    except FileNotFoundError:
        locationHackettstownServices1 = all_services[:100] if len(all_services) >= 100 else all_services
        with open("hackettstown_locations_services1.pkl", "wb") as f:
            pickle.dump(locationHackettstownServices1, f)
        print("Created services1 pickle from retrieved data.")

try:
    with open("hackettstown_locations_services2.pkl", "rb") as f:
        print("Loading services2 pickle...")
        locationHackettstownServices2 = pickle.load(f)
except FileNotFoundError:
    print("services2 pickle missing; ensuring we have services data...")
    if all_services is None:
        all_services = getBusinessLocations(soupHackettstownServices)
    # create services2 from remaining links (if any)
    locationHackettstownServices2 = all_services[100:] if len(all_services) > 100 else []
    with open("hackettstown_locations_services2.pkl", "wb") as f:
        pickle.dump(locationHackettstownServices2, f)
    print("Created services2 pickle from retrieved data.")

try:
    with open("hackettstown_locations_shopping.pkl", "rb") as f:
        print("Loading pickle...")
        locationHackettstownShopping = pickle.load(f)
except FileNotFoundError:
    print("Could not find file... attempting to retrieve data")
    locationHackettstownShopping = getBusinessLocations(soupHackettstownShopping)
    print("Data retrieved! now adding to collection")
    with open("hackettstown_locations_shopping.pkl", "wb") as f:
        pickle.dump(locationHackettstownShopping, f)

Loading pickle...
Loading pickle...
Loading pickle...
Could not find file... attempting to retreive data
Attempting session
Attempting Head
Attempt resp.url
1
Attempting session
Attempting Head
Attempt resp.url


KeyboardInterrupt: 

In [ ]:
with open("hackettstown_locations_restaurants.pkl", "rb") as f:
        print("Loading pickle...")
        locationHackettstownRestaurants = pickle.load(f)
print(getCoordinates(locationHackettstownRestaurants))



Loading pickle...
[('40.8670057', '-74.8289007'), ('40.8533252', '-74.8323381'), ('40.8548488', '-74.833719'), ('40.8352012', '-74.8251916'), ('40.8459396', '-74.8270394'), ('40.838083', '-74.8262833'), ('40.8533252', '-74.8323381'), ('40.838083', '-74.8262833'), ('40.8528594', '-74.8310392'), ('40.8545017', '-74.8360921'), ('40.8542771', '-74.8323818'), ('40.8417021', '-74.8269408'), ('40.8670057', '-74.8289061'), ('40.8369845', '-74.8261737'), ('40.8414401', '-74.8268537'), ('40.8536627', '-74.8313182'), ('40.8338189', '-74.8272934'), ('40.8528594', '-74.8310392'), ('40.8421262', '-74.8268985'), ('40.8532575', '-74.8304141'), ('40.8509413', '-74.8285747'), ('40.8528536', '-74.831013'), ('40.8543675', '-74.8326537'), ('40.8546286', '-74.8381297'), ('40.8418207', '-74.8268222'), ('40.8380212', '-74.8263654'), ('40.8520302', '-74.8295963'), ('40.8407413', '-74.8259034'), ('40.8548578', '-74.835799'), ('40.8528357', '-74.8303754'), ('40.8507595', '-74.8289789'), ('40.8542383', '-74.83252

In [ ]:
# Function to extract coordinates from a list of Google Maps URLs

def getCoordinates(locationHackettstown):
    coordsHackettstown = []

    #iterates through the lists of URLS
    for i in range(len(locationHackettstown)):  # List to store the coordinates

        #finds a match of the regex withen each item from locationHackettstown
        match = re.search(r'@(-?\d+(?:\.\d+)?),(-?\d+(?:\.\d+)?)', locationHackettstown[i])
        if not match:
            print(f"No coordinates found in URL: {locationHackettstown[i]}")
        if match:
            latitudeLocation = match.group(1) #makes a variable for the latitude based off of a regex grouping
            longitudeLocation = match.group(2) #makes a variable for the longitude based off of a regex grouping
        coordsHackettstown.append((latitudeLocation, longitudeLocation)) # appends the coordinates to the list as a tuple
        
    return coordsHackettstown


# Part 4: Creating a pandas Dataframe to store the location and business names

In [ ]:
# For this I will create indavidual dataframes for each category of location, further to combine the data later, I will be using data flattening methods.
namesHackettstownRestaurants = namesHackettstownRestaurants
dataRestaurants = pd.DataFrame({
    'Name': namesHackettstownRestaurants,
    'Latitude': [coord[0] for coord in coordinateHackettstownRestaurants],
    'Longitude': [coord[1] for coord in coordinateHackettstownRestaurants]
})
dataShopping = pd.DataFrame({
    'Name': namesHackettstownShopping,
    'Latitude': [coord[0] for coord in coordinateHackettstownShopping],
    'Longitude': [coord[1] for coord in coordinateHackettstownShopping]
})
dataServices = pd.DataFrame({
    'Name': namesHackettstownServices,
    'Latitude': [coord[0] for coord in coordinateHackettstownServices],
    'Longitude': [coord[1] for coord in coordinateHackettstownServices]
})
dataEntertainmentGaming = pd.DataFrame({
    'Name': namesHackettstownEntertainmentGaming,
    'Latitude': [coord[0] for coord in coordinateHackettstownEntertainmentGaming],
    'Longitude': [coord[1] for coord in coordinateHackettstownEntertainmentGaming]
})
dataCommunity = pd.DataFrame({
    'Name': namesHackettstownCommunity,
    'Latitude': [coord[0] for coord in coordinateHackettstownCommunity],
    'Longitude': [coord[1] for coord in coordinateHackettstownCommunity]
})

print(dataRestaurants)